In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604520,32.3,1292120,69.1,1292120,69.1
Vcells,1086848,8.3,8388608,64.0,1632674,12.5


In [2]:
semilla <- 100019
set.seed(semilla)

In [3]:
#Feature Engineering
#creo nuevas variables dentro del mismo mes
#Condimentar a gusto con nuevas variables

require("data.table")

#Establezco el Working Directory
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory

kcarpeta_datasets    <- "./datasets/"   #KAGGLE
#kcarpeta_datasets    <- "./buckets/b1/datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")
karchivo_salida      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_combPares.csv")



#lectura rapida del dataset  usando fread  de la libreria  data.table
dataset_orig  <- fread(karchivo_entrada)



Loading required package: data.table



In [4]:
EnriquecerDataset <- function( dataset )
{
  columnas_originales <-  copy(colnames( dataset ))


dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]

    
  dataset[ , minversion1                := rowSums( cbind( minversion1_pesos, minversion1_dolares) , na.rm=TRUE ) ]
    dataset[ , cinversion12                := rowSums( cbind( cinversion1, cinversion2) , na.rm=TRUE ) ]
    dataset[ , minversion12                := rowSums( cbind( minversion1, minversion2) , na.rm=TRUE ) ]
     dataset[ , tarj_credito_status       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
    dataset[ , tarj_credito_delinquency       := pmax( Master_delinquency,  Visa_delinquency, na.rm = TRUE) ]
    dataset[ , tarj_credito_mfinanciacion_limite    := rowSums( cbind(Master_mfinanciacion_limite,Visa_mfinanciacion_limite ) , na.rm=TRUE ) ]
    dataset[,dlg2 := ifelse(cliente_antiguedad==0,0,(mcomisiones/cliente_antiguedad))]
    dataset[,dlg3 := ifelse(mactivos_margen==0,0,(mcomisiones/mactivos_margen))]
    dataset[,dlg4 := ifelse(tcuentas==0,0,(ccuenta_corriente/tcuentas))]
    dataset[,dlg5 := ifelse(tcuentas==0,0,(tcuentas*tcuentas))]
    dataset[,dlg6 := ifelse(cliente_antiguedad==0,0,(mcuentas_saldo/cliente_antiguedad))]
  
    columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )

    return (dataset)
}
#------------------------------------------------------------------------------
dataset_enriq <- EnriquecerDataset( dataset_orig )


In [5]:
as.data.table(sapply(dataset_enriq[, list(mttarjeta_visa_debitos_automaticos)], sample, 10))

mttarjeta_visa_debitos_automaticos
<dbl>
2613.97
2985.28
10085.93
4852.26
36830.10
6014.69
10374.01
0.00
0.00


In [6]:
  #grabo con nombre extendido
  fwrite( dataset_enriq,
          file=karchivo_salida,
          sep= "," )